# Frontend Performance & Accessibility

A comprehensive guide to Core Web Vitals, performance optimization techniques, and accessibility standards.

## Topics Covered
1. Core Web Vitals (LCP, FID/INP, CLS)
2. Performance Optimization Techniques
3. Image Optimization
4. Caching Strategies
5. Accessibility (WCAG, ARIA, Semantic HTML)
6. Keyboard Navigation

## 1. Core Web Vitals

Core Web Vitals are Google's key metrics for measuring user experience.

### Metric Thresholds

| Metric | Good | Needs Improvement | Poor |
|--------|------|-------------------|------|
| **LCP** (Largest Contentful Paint) | ≤ 2.5s | 2.5s - 4.0s | > 4.0s |
| **INP** (Interaction to Next Paint) | ≤ 200ms | 200ms - 500ms | > 500ms |
| **CLS** (Cumulative Layout Shift) | ≤ 0.1 | 0.1 - 0.25 | > 0.25 |
| **FCP** (First Contentful Paint) | ≤ 1.8s | 1.8s - 3.0s | > 3.0s |
| **TTFB** (Time to First Byte) | ≤ 800ms | 800ms - 1800ms | > 1800ms |

### LCP - Largest Contentful Paint
Measures loading performance - when the largest content element becomes visible.

**Common LCP Elements:**
- `<img>` elements
- `<video>` poster images
- Elements with `background-image`
- Block-level text elements

**Optimization Strategies:**
- Preload critical resources: `<link rel="preload">`
- Optimize server response time
- Use CDN for static assets
- Remove render-blocking resources

### INP - Interaction to Next Paint (replaced FID)
Measures responsiveness - time from user interaction to visual feedback.

**Optimization Strategies:**
- Break up long tasks (> 50ms)
- Use `requestIdleCallback()` for non-critical work
- Minimize main thread work
- Use web workers for heavy computation

### CLS - Cumulative Layout Shift
Measures visual stability - unexpected layout shifts during page load.

**Common Causes & Fixes:**
- Images without dimensions → Always set `width` and `height`
- Ads/embeds → Reserve space with containers
- Web fonts → Use `font-display: swap` with fallbacks
- Dynamic content → Use `min-height` or skeleton screens

## 2. Performance Optimization Techniques

### Code Splitting
Divide code into smaller chunks loaded on demand.

```javascript
// Route-based splitting (React)
const Dashboard = React.lazy(() => import('./Dashboard'));
const Settings = React.lazy(() => import('./Settings'));

function App() {
  return (
    <Suspense fallback={<Loading />}>
      <Routes>
        <Route path="/dashboard" element={<Dashboard />} />
        <Route path="/settings" element={<Settings />} />
      </Routes>
    </Suspense>
  );
}

// Dynamic import with Webpack magic comments
const module = await import(
  /* webpackChunkName: "my-chunk" */
  /* webpackPrefetch: true */
  './heavyModule'
);
```

### Lazy Loading
Defer loading of non-critical resources until needed.

```html
<!-- Native lazy loading for images -->
<img src="image.jpg" loading="lazy" alt="Description">

<!-- Intersection Observer for custom lazy loading -->
<script>
const observer = new IntersectionObserver((entries) => {
  entries.forEach(entry => {
    if (entry.isIntersecting) {
      entry.target.src = entry.target.dataset.src;
      observer.unobserve(entry.target);
    }
  });
}, { rootMargin: '100px' });

document.querySelectorAll('img[data-src]').forEach(img => observer.observe(img));
</script>
```

### Tree Shaking
Eliminate dead code from bundles (requires ES modules).

```javascript
// ✅ Good - allows tree shaking
import { debounce } from 'lodash-es';

// ❌ Bad - imports entire library
import _ from 'lodash';

// package.json - mark package as side-effect free
{
  "sideEffects": false,
  // or specify files with side effects
  "sideEffects": ["*.css", "./src/polyfills.js"]
}
```

## 3. Image Optimization

### Modern Image Formats Comparison

| Format | Use Case | Compression | Browser Support |
|--------|----------|-------------|----------------|
| **WebP** | General use | 25-35% smaller than JPEG | 97%+ |
| **AVIF** | Best compression | 50% smaller than JPEG | 92%+ |
| **JPEG** | Photos, fallback | Lossy, good quality | 100% |
| **PNG** | Transparency, graphics | Lossless | 100% |
| **SVG** | Icons, logos, illustrations | Vector, scalable | 100% |

### Responsive Images

```html
<!-- srcset for resolution switching -->
<img 
  src="image-800.jpg"
  srcset="image-400.jpg 400w,
          image-800.jpg 800w,
          image-1200.jpg 1200w"
  sizes="(max-width: 600px) 100vw,
         (max-width: 1200px) 50vw,
         800px"
  alt="Responsive image"
  loading="lazy"
  decoding="async"
>

<!-- picture element for format fallbacks -->
<picture>
  <source srcset="image.avif" type="image/avif">
  <source srcset="image.webp" type="image/webp">
  <img src="image.jpg" alt="With format fallbacks">
</picture>
```

### Optimization Techniques
- **Compress images** - Use tools like Sharp, ImageOptim, Squoosh
- **Serve correct dimensions** - Don't resize in browser
- **Use CDN with transforms** - Cloudinary, Imgix, Cloudflare Images
- **Blur-up placeholders** - Low-quality image placeholders (LQIP)
- **Lazy load below fold** - Use `loading="lazy"` attribute

## 4. Caching Strategies

### HTTP Cache Headers

| Strategy | Cache-Control | Use Case |
|----------|---------------|----------|
| **Immutable assets** | `max-age=31536000, immutable` | Hashed files (app.a1b2c3.js) |
| **Mutable assets** | `no-cache` or `max-age=0, must-revalidate` | HTML, API responses |
| **Private data** | `private, max-age=0, no-store` | User-specific content |
| **Stale-while-revalidate** | `max-age=3600, stale-while-revalidate=86400` | Balance freshness & speed |

### Service Worker Caching

```javascript
// Cache-first strategy (for static assets)
self.addEventListener('fetch', (event) => {
  event.respondWith(
    caches.match(event.request).then((cached) => {
      return cached || fetch(event.request).then((response) => {
        const clone = response.clone();
        caches.open('v1').then(cache => cache.put(event.request, clone));
        return response;
      });
    })
  );
});

// Network-first strategy (for dynamic content)
self.addEventListener('fetch', (event) => {
  event.respondWith(
    fetch(event.request)
      .then(response => {
        caches.open('v1').then(cache => cache.put(event.request, response.clone()));
        return response;
      })
      .catch(() => caches.match(event.request))
  );
});
```

### Browser Storage Options

| Storage | Capacity | Persistence | Use Case |
|---------|----------|-------------|----------|
| **localStorage** | ~5MB | Permanent | User preferences, tokens |
| **sessionStorage** | ~5MB | Tab session | Temporary state |
| **IndexedDB** | Large (quota) | Permanent | Structured data, offline |
| **Cache API** | Large (quota) | Permanent | HTTP responses, assets |

## 5. Accessibility Fundamentals

### WCAG 2.1 Principles (POUR)

| Principle | Description | Examples |
|-----------|-------------|----------|
| **Perceivable** | Content must be presentable | Alt text, captions, color contrast |
| **Operable** | UI must be operable | Keyboard access, skip links, no seizures |
| **Understandable** | Content must be understandable | Clear language, predictable UI |
| **Robust** | Content works with assistive tech | Valid HTML, ARIA when needed |

### WCAG Conformance Levels

| Level | Description | Target |
|-------|-------------|--------|
| **A** | Minimum accessibility | Basic requirements |
| **AA** | Acceptable accessibility | Legal standard (most regulations) |
| **AAA** | Optimal accessibility | Specialized content |

### Color Contrast Requirements

| Text Type | AA Ratio | AAA Ratio |
|-----------|----------|----------|
| Normal text (< 18px) | 4.5:1 | 7:1 |
| Large text (≥ 18px or 14px bold) | 3:1 | 4.5:1 |
| UI components & graphics | 3:1 | N/A |

### Semantic HTML

```html
<!-- ✅ Good - semantic structure -->
<header>
  <nav aria-label="Main navigation">
    <ul>
      <li><a href="/">Home</a></li>
      <li><a href="/about">About</a></li>
    </ul>
  </nav>
</header>

<main>
  <article>
    <h1>Page Title</h1>
    <section aria-labelledby="intro">
      <h2 id="intro">Introduction</h2>
      <p>Content here...</p>
    </section>
  </article>
  <aside aria-label="Related content">...</aside>
</main>

<footer>...</footer>

<!-- ❌ Bad - div soup -->
<div class="header">
  <div class="nav">...</div>
</div>
```

## 6. ARIA Essentials

### Key ARIA Rules
1. **Don't use ARIA if native HTML works** - `<button>` over `<div role="button">`
2. **Don't change native semantics** - Don't add `role="heading"` to `<h1>`
3. **All interactive ARIA elements must be keyboard accessible**
4. **Don't hide focusable elements** - No `aria-hidden="true"` on focusable items
5. **All interactive elements need accessible names**

### Common ARIA Patterns

```html
<!-- Labeling -->
<button aria-label="Close dialog">×</button>
<input aria-labelledby="label-id" aria-describedby="hint-id">

<!-- Live regions (for dynamic updates) -->
<div aria-live="polite" aria-atomic="true">
  <!-- Screen reader announces changes -->
  3 items added to cart
</div>
<div role="alert">Error: Invalid email</div>
<div role="status">Loading complete</div>

<!-- Expanded/collapsed -->
<button aria-expanded="false" aria-controls="menu">
  Menu
</button>
<ul id="menu" hidden>...</ul>

<!-- Modal dialogs -->
<div role="dialog" aria-modal="true" aria-labelledby="dialog-title">
  <h2 id="dialog-title">Confirm Action</h2>
  <p>Are you sure?</p>
  <button>Confirm</button>
  <button>Cancel</button>
</div>

<!-- Form validation -->
<input 
  type="email" 
  aria-invalid="true" 
  aria-errormessage="email-error"
  aria-required="true">
<span id="email-error" role="alert">Please enter a valid email</span>
```

### ARIA Roles Reference

| Category | Roles |
|----------|-------|
| **Landmark** | `banner`, `main`, `navigation`, `complementary`, `contentinfo`, `search` |
| **Widget** | `button`, `checkbox`, `dialog`, `menu`, `tab`, `tabpanel`, `tooltip` |
| **Live Region** | `alert`, `log`, `marquee`, `status`, `timer` |

## 7. Keyboard Navigation

### Focus Management

```html
<!-- Skip link implementation -->
<a href="#main-content" class="skip-link">Skip to main content</a>

<style>
.skip-link {
  position: absolute;
  top: -40px;
  left: 0;
  padding: 8px;
  background: #000;
  color: #fff;
  z-index: 100;
}
.skip-link:focus {
  top: 0;
}
</style>
```

```javascript
// Focus trap for modals
function trapFocus(modal) {
  const focusable = modal.querySelectorAll(
    'button, [href], input, select, textarea, [tabindex]:not([tabindex="-1"])'
  );
  const first = focusable[0];
  const last = focusable[focusable.length - 1];

  modal.addEventListener('keydown', (e) => {
    if (e.key === 'Tab') {
      if (e.shiftKey && document.activeElement === first) {
        e.preventDefault();
        last.focus();
      } else if (!e.shiftKey && document.activeElement === last) {
        e.preventDefault();
        first.focus();
      }
    }
    if (e.key === 'Escape') closeModal();
  });
}
```

```css
/* Visible focus styles */
:focus-visible {
  outline: 2px solid #005fcc;
  outline-offset: 2px;
}
```

### Keyboard Shortcuts Reference

| Key | Action |
|-----|--------|
| `Tab` | Move to next focusable element |
| `Shift + Tab` | Move to previous focusable element |
| `Enter` / `Space` | Activate buttons, links |
| `Arrow keys` | Navigate within components (tabs, menus, radios) |
| `Escape` | Close modals, dropdowns |
| `Home` / `End` | Jump to first/last item |

### Roving Tabindex Pattern

```javascript
// For tab panels, menus, toolbars - only one item is tabbable
const items = document.querySelectorAll('[role="tab"]');
let currentIndex = 0;

items.forEach((item, index) => {
  item.tabIndex = index === 0 ? 0 : -1;
  
  item.addEventListener('keydown', (e) => {
    if (e.key === 'ArrowRight') {
      items[currentIndex].tabIndex = -1;
      currentIndex = (currentIndex + 1) % items.length;
      items[currentIndex].tabIndex = 0;
      items[currentIndex].focus();
    }
  });
});
```

## Quick Reference Checklist

### Performance Checklist
- [ ] LCP < 2.5s, INP < 200ms, CLS < 0.1
- [ ] Code splitting by route/feature
- [ ] Images: WebP/AVIF, responsive, lazy loaded
- [ ] Cache immutable assets with long max-age
- [ ] Preload critical resources
- [ ] Minimize JavaScript bundle size

### Accessibility Checklist
- [ ] All images have descriptive alt text
- [ ] Color contrast meets WCAG AA (4.5:1 normal, 3:1 large)
- [ ] All functionality keyboard accessible
- [ ] Focus indicators visible
- [ ] Skip link to main content
- [ ] Proper heading hierarchy (h1 → h2 → h3)
- [ ] Form inputs have labels
- [ ] Error messages are associated with inputs
- [ ] ARIA used appropriately (prefer native HTML)
- [ ] Tested with screen reader

### Testing Tools
| Tool | Purpose |
|------|---------|
| **Lighthouse** | Performance & accessibility audits |
| **axe DevTools** | Accessibility testing |
| **WebPageTest** | Detailed performance analysis |
| **WAVE** | Web accessibility evaluation |
| **Chrome DevTools** | Network, Performance, Accessibility tabs |